<a href="https://colab.research.google.com/github/fazaghifari/BayesianOptTools/blob/newver/Single_Objective_Bayesian_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Single Objective Bayesian Optimization

In this notebook we address a surrogate-assisted single objective bayesian optimization (SOBO) problem with Gaussian Process/Kriging as the surrogate model. In this tutorial, we will try to minimize Styblinski-Tang function. KADAL will be used as the main framework.

First, let's install the packages.

In [0]:
!pip install git+https://github.com/flowdiagnosticsitb/KADAL.git
!pip install cma

  Cloning https://github.com/flowdiagnosticsitb/KADAL.git to /tmp/pip-req-build-49ul8efb
  Running command git clone -q https://github.com/flowdiagnosticsitb/KADAL.git /tmp/pip-req-build-49ul8efb
     |████████████████████████████████| 204kB 2.7MB/s 
     |████████████████████████████████| 686kB 46.9MB/s 
     |████████████████████████████████| 419kB 50.3MB/s 
     |████████████████████████████████| 204kB 49.3MB/s 
  Created wheel for kadal: filename=kadal-1.0.2-cp36-none-any.whl size=75807 sha256=fa7037b7bffa2f5bc7828aad0796f926e386dffba644a3e9867da01de1833246
  Stored in directory: /tmp/pip-ephem-wheel-cache-x0z4j4q0/wheels/3c/fd/38/70e82ea24ca6ec25615628dafcc6174b1ba9f859aa5e3f237b
  Created wheel for configobj: filename=configobj-5.0.6-cp36-none-any.whl size=34546 sha256=f90eaedabfc458e6fcc4ca10c8451e3a1a4b0c7245a6c612f625f0984659552f
  Stored in directory: /root/.cache/pip/wheels/f1/e4/16/4981ca97c2d65106b49861e0b35e2660695be7219a2d351ee0
Successfully built kadal configobj
ERROR: 

## Import KADAL

In [0]:
import numpy as np
from kadal.testcase.analyticalfcn.cases import evaluate
from kadal.surrogate_models.kriging_model import Kriging
from kadal.surrogate_models.supports.initinfo import initkriginfo
from kadal.misc.sampling.samplingplan import sampling
from kadal.optim_tools.SOBO import SOBO
import time

### Create Kriging Function
Now, let's create a function that handle Kriging training process. In this demo we will use Styblinski-Tang fucntion with 40 samples as a test case.

In [0]:
def construct_krig():
    nsample = 10
    nvar = 2
    ub = np.array([5, 5])  # Define upper bound for the problem
    lb = np.array([-5, -5])  # Define lower bound for the problem

    # Now generate the sample points using "sampling" function
    # In this case we use Halton sampling
    # We also want to return "real" value of the sampling point, instead of
    # the normalized samples, hence we include result = 'real in the argument 
    samplenorm, X = sampling("halton", nvar, nsample, result='real',
                                  upbound=ub, lobound=lb)
    
    # Then, the samples are evaluated. In this case we use the built-in test 
    # case in KADAL.
    y = evaluate(X, 'styblinski')

    # Kriging in KADAL use a dictionary as its main information container
    # To initialize the dictionary:
    KrigInfo = initkriginfo()
    # Set KrigInfo
    KrigInfo['X'] = X  # KrigInfo['X'] and ['y'] is mandatory
    KrigInfo['y'] = y  
    KrigInfo['nrestart'] = 5  # Set the number of restarting points for hyperparameter optimization
    KrigInfo['ub'] = ub  # Set the upper bound of the problem, if empty maximum value of the sample is taken
    KrigInfo['lb'] = lb  # Same with ub
    KrigInfo["kernel"] = ["gaussian"]  # We support multi-kernel, e.g ["gaussian", "matern32"]
    KrigInfo["optimizer"] = "lbfgsb" # Use L-BFGS-B as the hyperparam optimizer
    KrigInfo["problem"] = "styblinski"  # Define the function problem, to be evaluated in the SOBO subroutine

    # Construct Kriging
    t = time.time()
    # Now, initialize the Kriging object. Set the sample standardization into True
    # But, we don't want to normalize the response (normy = False)
    # We also don't want to train the Kriging variance ('trainvar = False')
    krigobj = Kriging(KrigInfo, standardization=True)
    # Train Kriging
    krigobj.train()
    # Calculate the leave-one-out cross validation (optional)
    loocverr,_ = krigobj.loocvcalc(metrictype='rmse')
    elapsed = time.time() - t
    print("elapsed time for train Kriging model: ", elapsed, "s")
    print("LOOCV error of Kriging model: ", loocverr, "%")

    return krigobj


### Create optimizer function handler

In [0]:
def runopt(krigobj):
    # Similar with Kriging, SOBO take dictionary as its input
    soboInfo = dict()
    soboInfo['nup'] = 50  # Set maximum number of update
    soboInfo['stalliteration'] = 10  # Set stall number (if the minimum value is still the same after n times, the optimizer stop)
    soboInfo['nrestart'] = 3  # Number of restarting point for optimizer
    soboInfo['acquifunc'] = 'EI'  # Set acquisition function to Expected Improvement (EI)
    soboInfo['acquifuncopt'] = 'diff_evo'  # Set acquisition function optimizer to differential evolution

    # Initialize SOBO object. Set autoupdate to true, to make the program automatically evaluate the problem
    optim = SOBO(soboInfo,krigobj,autoupdate=True)
    x_optim,y_optim = optim.run()
    return x_optim,y_optim

## Run

In [0]:
kriging_model = construct_krig()
X_optimum, y_optimum = runopt(kriging_model)

Begin train hyperparam.
Training 5 hyperparameter(s)
Training hyperparameter candidate no.1
Training hyperparameter candidate no.2
Training hyperparameter candidate no.3
Training hyperparameter candidate no.4
Training hyperparameter candidate no.5
Single Objective, train hyperparam, end.
Best hyperparameter is [-1.25 -1.25]
With NegLnLikelihood of 42.76734625432907
elapsed time for train Kriging model:  0.2396831512451172 s
LOOCV error of Kriging model:  80.00548066291594 %
The number of stall iteration is specified to  10  by user
The number of restart for acquisition function optimization is specified to  3  by user
The file name for saving the results is not specified, set the name to temporarydata.mat
Begin single-objective Bayesian optimization process.
Update no.: 1, F-count: 10, Best f(x): -53.30842734291648, Stall counter: 0
Update no.: 2, F-count: 10, Best f(x): [-53.30842734], Stall counter: 1
Update no.: 3, F-count: 10, Best f(x): [-56.96118495], Stall counter: 0
Update no.:

In [0]:
print("Aprrox. optimum value is: ",y_optimum," at X= ", X_optimum)

Aprrox. optimum value is:  -78.33207079449447  at X=  [-2.90117521 -2.9066166 ]


Real optimum value for 2D Styblinski-Tang should be -78.332 at X = [-2.903534, -2.903534]